In [115]:
import pandas as pd
import numpy as np
import urllib.request, json
from pandas.io.json import json_normalize

# записать JSON объект с сайта по id матчу
def get_match_on_id(ID, site_name='https://api.opendota.com/api/matches/'):
    #название сайта для одного матча
    site = site_name + str(ID)
    #Прочитаь JSON с сайта для одного матча
    with urllib.request.urlopen(site) as url:
        return json.loads(url.read().decode())

# Вытащить данные из json объекта по ключу в виде датафрейма
def get_from_JSON_for_key(data, key):
    return pd.DataFrame(data.get(key))

# Вытащить имя команды из объекта json по их ID
def get_teamName(data, dire_or_radiant):
    team_id = data.get(dire_or_radiant + '_team').get('team_id')
    data_name_teams = get_match_on_id(team_id, 'https://api.opendota.com/api/teams/')
    return data_name_teams.get("name")

# Вытащить ID команды из моего файла НАПИСАТЬ ФУНКЦИЮ
def team_id(my_file, team_name, not_correct_team_name, ID):
    team_id = my_file.get(team_name)
    if team_id == None:
        not_correct_team_name.append([ID, team_name])
    return team_id

# Создать DF по пикам и банам из матча
def pick_ban(data, result):
    # Первый пик радиант
    first_ban_radiant = (int)(not bool(result['team'][0]))

    # Запись всех пиков и банов подряд из DF по пикам и банам в массив
    a =[first_ban_radiant]
    for id_hero in result['hero_id']:
        a.append(id_hero)

#     Название столбцов в DF пики и баны для патча 7,07+
#     name_columns_picks_bans =['first_ban_radiant', 'ban1', 'ban2','ban3', 'ban4', 'ban5', 'ban6', 'pick1', 'pick2', 'pick3', 'pick4', 
#                               'ban7', 'ban8','ban9', 'ban10', 'pick5', 'pick6', 'pick7', 'pick8', 
#                               'ban11', 'ban12', 'pick9', 'pick10']
    
    # Название столбцов в DF пики и баны для патча 7,06+
    name_columns_picks_bans =['first_ban_radiant', 'ban1', 'ban2','ban3', 'ban4', 'pick1', 'pick2', 'pick3', 'pick4', 
                              'ban7', 'ban8','ban9', 'ban10', 'pick5', 'pick6', 'pick7', 'pick8', 
                              'ban11', 'ban12', 'pick9', 'pick10']
    # создание DF для пиков и банов с правильным названием столбцов
    picks_bans = pd.DataFrame([a], columns=name_columns_picks_bans)
    return picks_bans

In [134]:
# Прочитать файл с id матчами
matches_path = pd.read_csv('id for 7.06.csv')
#matches_path707 = pd.read_csv('test.csv')
# Прочитать файл с нормальными id команд
file_name_and_ID_team = json.load(open('ID_nameTeam'))
# Список с командами не из моего списка команд с id
name_teams_not_with_myID = []
# Основной DF для всех матчей
matches_df = pd.DataFrame()

In [136]:
for ID in matches_path['match_id']:
    try:
        data = get_match_on_id(ID)
        # Выбрать DF по Picks_bans (пики и баны для матча 3534749496)
        result_with_picks_bans = get_from_JSON_for_key(data, 'picks_bans')
        try:

    #         #Для патсча7,07+
    #         if len(result_with_picks_bans['hero_id']) == 22:
            # Для патча7,06
            if len(result_with_picks_bans['hero_id']) == 20:    
                # Название и id лиги
                league = data.get('league')
                league_name = league.get('name')
                league_id = league.get('leagueid')

                #Имена команд
                radiant_name = get_teamName(data, 'radiant')
                dire_name = get_teamName(data, 'dire')

                # id команд и запись команд, отсутсвующих в моем файле, в отдельный список
                radiant_id = team_id(file_name_and_ID_team, radiant_name, name_teams_not_with_myID, ID)
                dire_id = team_id(file_name_and_ID_team, dire_name, name_teams_not_with_myID, ID)

                # победили редиант
                radiant_win = (int)(data.get('players').pop().get('radiant_win'))

                #пики и баны команд, кто первый пикнул
                picks_bans = pick_ban(data, result_with_picks_bans)

                # Версия патча
                version_id = data.get('version')

                # Кол-во убийсвт в матче
                dire_score = data.get('dire_score')
                radiant_score = data.get('radiant_score')

                # регион
                region = data.get('region')
                table_for_match = pd.DataFrame([[ID, version_id, region,  league_name, league_id, radiant_name, radiant_id, 
                                                 dire_name, dire_id, radiant_score, dire_score]], 
                                    columns=['id matches', 'patch', 'region', 'league_name', 'league_id', 'radiant_name', 
                                             'radiant_id', 'dire_name', 'dire_id', 'radiant_score', 'dire_score'])
                df = pd.merge(table_for_match, picks_bans, left_index=True, right_index=True)
                df['radiant_win'] = radiant_win
                matches_df = matches_df.append(df)
        except KeyError:
            print('Отсутсвуют id героев. ID матча - ', ID)
        except AttributeError:
            print ('Отсутсвуют id команды. ID матча - ', ID)
    except Exception as e :
        print ('ID матча - ', ID)
        print (e)

Отсутсвуют id команды. ID матча -  3186842141
Отсутсвуют id команды. ID матча -  3186952395
Отсутсвуют id команды. ID матча -  3187234555
Отсутсвуют id команды. ID матча -  3187367388
Отсутсвуют id команды. ID матча -  3187548218
Отсутсвуют id команды. ID матча -  3187627651
Отсутсвуют id команды. ID матча -  3190891171
Отсутсвуют id команды. ID матча -  3191027660
Отсутсвуют id команды. ID матча -  3191177071
Отсутсвуют id команды. ID матча -  3195526598
Отсутсвуют id команды. ID матча -  3195675077
Отсутсвуют id команды. ID матча -  3197798103
Отсутсвуют id команды. ID матча -  3197916872
Отсутсвуют id команды. ID матча -  3198028468
Отсутсвуют id команды. ID матча -  3199783328
Отсутсвуют id команды. ID матча -  3199916628
Отсутсвуют id команды. ID матча -  3199994868
Отсутсвуют id команды. ID матча -  3201318018
Отсутсвуют id команды. ID матча -  3201411639
Отсутсвуют id команды. ID матча -  3201568410
Отсутсвуют id команды. ID матча -  3201738393
Отсутсвуют id команды. ID матча - 

Отсутсвуют id команды. ID матча -  3456358330
Отсутсвуют id команды. ID матча -  3456361675
Отсутсвуют id команды. ID матча -  3456365966
Отсутсвуют id команды. ID матча -  3456372960
Отсутсвуют id команды. ID матча -  3456374574
Отсутсвуют id команды. ID матча -  3456523320
Отсутсвуют id команды. ID матча -  3456525320
Отсутсвуют id команды. ID матча -  3456525747
Отсутсвуют id команды. ID матча -  3456525750
Отсутсвуют id команды. ID матча -  3456533586
Отсутсвуют id команды. ID матча -  3456535005
Отсутсвуют id команды. ID матча -  3456536923
Отсутсвуют id команды. ID матча -  3456538293
Отсутсвуют id команды. ID матча -  3456541216
Отсутсвуют id команды. ID матча -  3456541992
Отсутсвуют id команды. ID матча -  3456695265
Отсутсвуют id команды. ID матча -  3456695277
Отсутсвуют id команды. ID матча -  3456697211
Отсутсвуют id команды. ID матча -  3456697943
Отсутсвуют id команды. ID матча -  3456698887
Отсутсвуют id команды. ID матча -  3456699977
Отсутсвуют id героев. ID матча -  

In [139]:
matches_df.to_csv('table with all matches patch 7.06.csv')

In [138]:
len(name_teams_not_with_myID)

3257